In [1]:
!pip install langchain chromadb sentence-transformers pymupdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 10.6 MB/s eta 0

In [2]:
!pip install -q pypdf langchain openai faiss-cpu tiktoken sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.6 MB/s eta 0:00:00


In [3]:
!pip install -q "unstructured[pdf]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 22.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.9/527.9 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━

In [4]:
!pip install -q gradio

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
pdf_folder_path = '/content/drive/MyDrive/LexAi_pdfs'

In [7]:
!pip install -q langchain-community unstructured pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00


In [9]:
import os
from pypdf import PdfReader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_pdfs_from_folder(folder_path):
    documents = []
    for file in os.listdir(folder_path):
        if file.endswith('.pdf'):
            file_path = os.path.join(folder_path, file)
            try:
                # Try structured PDF first
                reader = PdfReader(file_path)
                text = ""
                for page in reader.pages:
                    text += page.extract_text()
                if len(text) > 100:  # If we got reasonable text
                    documents.append({
                        'source': file,
                        'text': text,
                        'pages': len(reader.pages)
                    })
                else:
                    # Fall back to unstructured loader
                    loader = UnstructuredPDFLoader(file_path)
                    data = loader.load()
                    documents.append({
                        'source': file,
                        'text': data[0].page_content,
                        'pages': 'unknown'
                    })
            except Exception as e:
                print(f"Error processing {file}: {str(e)}")
    return documents

# Load all PDFs
legal_docs = load_pdfs_from_folder(pdf_folder_path)
print(f"Loaded {len(legal_docs)} PDF documents")

Loaded 11 PDF documents


In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", "(?<=\\. )", " ", ""]
)

document_chunks = []
for doc in legal_docs:
    chunks = text_splitter.split_text(doc['text'])
    for i, chunk in enumerate(chunks):
        document_chunks.append({
            'text': chunk,
            'source': doc['source'],
            'chunk_num': i+1,
            'metadata': {
                'pages': doc['pages'],
                'file_name': doc['source']
            }
        })

print(f"Created {len(document_chunks)} text chunks from {len(legal_docs)} documents")

Created 13203 text chunks from 11 documents


In [12]:
!pip install -q sentence-transformers chromadb langchain-chroma

In [13]:
!pip install -q langchain-huggingface sentence-transformers

In [14]:
# Step 5: Embed the chunks
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
# Step 6: Convert chunks to LangChain Documents
from langchain_core.documents import Document

documents = [
    Document(
        page_content=chunk["text"],
        metadata={"source": chunk["source"], "chunk_num": chunk["chunk_num"]}
    ) for chunk in document_chunks
]

In [16]:
!pip install -q chromadb langchain-chroma

In [17]:
# Step 7: Create Chroma vector DB
from langchain_chroma import Chroma

vector_db = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory="./legal_docs_chromadb"
)

print(f"Created ChromaDB with {len(documents)} legal document chunks!")


Created ChromaDB with 13203 legal document chunks!


In [18]:
# Step 8: Create Retriever with top-k filtering
retriever = vector_db.as_retriever(search_kwargs={
    "k": 3
})

In [19]:
!pip install -q transformers torch

In [20]:
# Step 9: Load LLM (Flan-T5)
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

hf_pipe = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    device="cpu"
)

llm = HuggingFacePipeline(pipeline=hf_pipe)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
/tmp/ipython-input-20-1511719320.py:12: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipe)


In [21]:
# Step 10: Create RetrievalQA Chain
legal_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [22]:
# Step 11: Ask Legal Question
query = "What is the punishment under Section 302 of the PPC?"
result = legal_qa.invoke({"query": query})

print("\n⚖️ Legal Answer:")
print(result["result"])

print("\n📚 Source Documents:")
for doc in result["source_documents"]:
    print(f"- {doc.metadata['source']} (Chunk {doc.metadata['chunk_num']})")



⚖️ Legal Answer:
Whether the police may arrest without warrant or not Whether a warrant or a summon shall ordinarily be issued in the first instance. Whether bailable or not whether compoundable or not Punishment under the PPC By what Court triable

📚 Source Documents:
- Code_of_criminal_procedure_1898.pdf (Chunk 67)
- Code_of_criminal_procedure_1898.pdf (Chunk 1217)
- Code_of_criminal_procedure_1898.pdf (Chunk 1216)


In [23]:
import shutil

# Save and zip the vector DB folder
shutil.make_archive('/content/legal_docs_chromadb', 'zip', './legal_docs_chromadb')


'/content/legal_docs_chromadb.zip'

In [24]:
from google.colab import files
files.download('/content/legal_docs_chromadb.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>